In [2]:
import numpy as np
import os
from keras.layers import Conv3D, MaxPooling3D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
#from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.preprocessing import LabelEncoder
import sklearn
import seaborn as sns
import tensorflow as tf
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from prettytable import PrettyTable
from matplotlib import pyplot as plt

import time

# Resnet 50

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import ResNet50

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 17
TRAIN_DATA_DIR = "D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\output_denoised_image_folder"

# Define data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load and preprocess the data with splitting
train_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  # 80% of the data will be used for training
)

validation_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # 20% of the data will be used for validation
)



Found 246 images belonging to 17 classes.
Found 52 images belonging to 17 classes.


In [3]:

# Define the ResNet50 model
model = ResNet50(include_top=False, weights='imagenet', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Add new layers to the model
x = model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Combine the original model with the new layers
model = tf.keras.models.Model(inputs=model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Start the timer
start_time = time.time()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# Stop the timer
end_time = time.time()

# Compute the computational time
comp_time = end_time - start_time

# Evaluate the model
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(validation_generator)

print(f'Training Accuracy: {train_acc * 100}')
print(f'Validation Accuracy: {val_acc * 100}')
print(f'Trainig time: {comp_time}')

# Save the trained model
model.save('denoised_resnet_50.h5')




Epoch 1/100


15/15 [==============================] - 27s 822ms/step - loss: 4.9895 - accuracy: 0.1870 - val_loss: 21.6023 - val_accuracy: 0.0833
Epoch 2/100
15/15 [==============================] - 11s 750ms/step - loss: 2.3308 - accuracy: 0.4000 - val_loss: 3.7267 - val_accuracy: 0.0833
Epoch 3/100
15/15 [==============================] - 11s 756ms/step - loss: 1.6619 - accuracy: 0.4870 - val_loss: 4.0868 - val_accuracy: 0.0417
Epoch 4/100
15/15 [==============================] - 11s 752ms/step - loss: 1.1394 - accuracy: 0.6435 - val_loss: 2.8318 - val_accuracy: 0.0417
Epoch 5/100
15/15 [==============================] - 11s 772ms/step - loss: 0.8955 - accuracy: 0.7522 - val_loss: 3.8461 - val_accuracy: 0.0417
Epoch 6/100
15/15 [==============================] - 11s 754ms/step - loss: 0.8680 - accuracy: 0.7826 - val_loss: 4.7785 - val_accuracy: 0.0625
Epoch 7/100
15/15 [==============================] - 11s 765ms/step - loss: 0.6905 - accuracy: 0.7783 - val_loss: 6.1572 - val_acc

c:\Users\ksaqu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Resnet Mobile

In [9]:
import tensorflow as tf
import time  # Import the time module
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import MobileNetV2

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 17
TRAIN_DATA_DIR = "D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\output_denoised_image_folder"

# Define data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load and preprocess the data with splitting
train_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  # 80% of the data will be used for training
)

validation_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # 20% of the data will be used for validation
)

# Define the MobileNetV2 model
base_model = MobileNetV2(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
                         include_top=False,
                         weights='imagenet')

# Freeze the layers in the base model
base_model.trainable = False

# Create new model on top
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Start the timer
start_time = time.time()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# Stop the timer
end_time = time.time()

# Compute the computational time
comp_time = end_time - start_time

# Evaluate the model
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(validation_generator)

model.save('denoised_Resnetmobile.h5')

print(f'Training Accuracy: {train_acc * 100}')
print(f'Validation Accuracy: {val_acc * 100}')
print(f'Training time: {comp_time} seconds')


Found 246 images belonging to 17 classes.
Found 52 images belonging to 17 classes.
Epoch 1/100
15/15 [==============================] - 4s 125ms/step - loss: 17.8204 - accuracy: 0.0957 - val_loss: 8.4056 - val_accuracy: 0.1042
Epoch 2/100
15/15 [==============================] - 1s 66ms/step - loss: 4.5328 - accuracy: 0.2652 - val_loss: 2.5651 - val_accuracy: 0.2292
Epoch 3/100
15/15 [==============================] - 1s 66ms/step - loss: 1.8647 - accuracy: 0.4609 - val_loss: 2.0776 - val_accuracy: 0.3958
Epoch 4/100
15/15 [==============================] - 1s 65ms/step - loss: 1.4250 - accuracy: 0.5130 - val_loss: 1.7508 - val_accuracy: 0.4792
Epoch 5/100
15/15 [==============================] - 1s 67ms/step - loss: 1.0695 - accuracy: 0.7042 - val_loss: 1.8100 - val_accuracy: 0.4375
Epoch 6/100
15/15 [==============================] - 1s 66ms/step - loss: 0.8890 - accuracy: 0.7435 - val_loss: 1.1989 - val_accuracy: 0.6250
Epoch 7/100
15/15 [==============================] - 1s 71ms/st

# ResNet-Mobile and VGG

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Concatenate
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.optimizers import Adam

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 17
TRAIN_DATA_DIR = "D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\output_denoised_image_folder"

# Define data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load and preprocess the data with splitting
train_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  # 80% of the data will be used for training
)

validation_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # 20% of the data will be used for validation
)

# Define the MobileNetV2 model
mobilenet_model = MobileNetV2(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
                              include_top=False,
                              weights='imagenet')

# Define the VGG16 model
vgg16_model = VGG16(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
                    include_top=False,
                    weights='imagenet')

# Freeze the layers in both base models
mobilenet_model.trainable = False
vgg16_model.trainable = False

# Define input layer
input_layer = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Obtain outputs from both models
mobilenet_output = mobilenet_model(input_layer)
vgg16_output = vgg16_model(input_layer)

# Flatten the outputs
mobilenet_flatten = Flatten()(mobilenet_output)
vgg16_flatten = Flatten()(vgg16_output)

# Concatenate the outputs
concatenated = Concatenate()([mobilenet_flatten, vgg16_flatten])

# Add dense layers for classification
x = Dense(128, activation='relu')(concatenated)
output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the model
combined_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
combined_model.compile(optimizer=Adam(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Train the model
history = combined_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# Evaluate the model
train_loss, train_acc = combined_model.evaluate(train_generator)
val_loss, val_acc = combined_model.evaluate(validation_generator)

# Save the model
combined_model.save('denoised_resnet_vgg_combined.h5')

print(f'Training Accuracy: {train_acc * 100}')
print(f'Validation Accuracy: {val_acc * 100}')
print(f'Training time: {comp_time} seconds')


Found 246 images belonging to 17 classes.
Found 52 images belonging to 17 classes.
Epoch 1/100
15/15 [==============================] - 10s 448ms/step - loss: 19.3752 - accuracy: 0.0913 - val_loss: 12.2402 - val_accuracy: 0.0417
Epoch 2/100
15/15 [==============================] - 6s 386ms/step - loss: 6.5794 - accuracy: 0.2565 - val_loss: 3.8653 - val_accuracy: 0.3542
Epoch 3/100
15/15 [==============================] - 6s 429ms/step - loss: 2.2754 - accuracy: 0.3174 - val_loss: 2.0926 - val_accuracy: 0.2917
Epoch 4/100
15/15 [==============================] - 6s 401ms/step - loss: 1.6225 - accuracy: 0.5130 - val_loss: 1.8274 - val_accuracy: 0.5208
Epoch 5/100
15/15 [==============================] - 6s 379ms/step - loss: 1.2598 - accuracy: 0.6391 - val_loss: 1.7819 - val_accuracy: 0.5000
Epoch 6/100
15/15 [==============================] - 5s 344ms/step - loss: 0.9263 - accuracy: 0.7304 - val_loss: 1.3524 - val_accuracy: 0.6250
Epoch 7/100
15/15 [==============================] - 5s 